<a href="https://colab.research.google.com/github/BP-Development2024/Schichtaufbauten-Generator/blob/main/Schichtaufbauten_Generator_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wichtige Parameter, Materialdaten & Bauteileigenschaften als Array:
Hier können die Randbedingungen für die Erzeugung der Bauteildatenbank eingetragen werden. Auch findet sich hier die Definition der Größe der zu erzeugenden Datenbank und die zugrundeliegende Materialdatenbank, welche erweitert werden kann.

In [1]:
Anzahl_Bauteile = 10000 # Wie viele Bauteile mit korrespondierenden Eigenschaften sollen generiert werden?
Schichten = 6 # Anzahl der Schichten, die ein Bauteil maximal haben soll
min_Schichten = 3 # Anzahl der Schichten, die ein Bauteil minimal haben soll
# Maximal sinnvolle Werte für Normalisierung & Generierung halbwegs realistischer Aufbauten
max_Schichtdicke = 0.5 # typisch: 0.5
min_Schichtdicke = 0.0001 # typ. 0.0001
max_Dicke = 1.0 # typ. 1.0
min_Dicke = 0.05 # typ. 0.05
max_Gewicht = 1400 # typ. 1400
max_Tragfaehigkeit = 250 # Wert pro forma, Beschränkung der Tragfähigkeit macht nicht wirklich Sinn
min_Tragfaehigkeit = 0.1 # typ. 0.1
max_Uwert = 2 # typ. 2
max_GWP = 400 # typ. 400
max_Kosten = 400 # typ. 400
max_Waermekapazitaet = 10000000 # Wert pro forma, Beschränkung der Wärmekapazität macht nicht wirklich Sinn
max_Werte = [max_Dicke, max_Gewicht, max_Tragfaehigkeit, max_Uwert, max_GWP, max_Kosten, 1, 1, 1, max_Waermekapazitaet]
Feuchte_beachten = True # sollen nur Bauteilaufbauten generiert werden, bei denen keine Feuchte anfällt? (Randbedingungen siehe unten)
Oberflaechen_beachten = True # sollen nur Bauteilaufbauten generiert werden, die übliche Oberflächenmaterialien haben? (z.B. Putz, GK, Holz)
Innenaufbau_beachten = True # sollen nur Bauteilaufbauten generiert werden, die keine unüblichen Materialien im Inneren enthalten (z.B. kein Putz im Inneren der Wand)

# Definition der Bedingungen für die Feuchteermittlung nach Glaser-Verfahren
Innentemperatur = 20 # °C
Aussentemperatur = -5 # °C
RLF_i = 50  # Relative Luftfeuchtigkeit innen [%]
RLF_a = 80  # Relative Luftfeuchtigkeit aussen [%]

# Es folgt die zugrundeliegende Materialdatenbank
# Dichte [kg/m³], min. Dicke [m], max. Dicke [m], Druckfestigkeit [MN/m2], Wärmeleitfähigkeit [W/mK], Wärmeübergangswiderstand innen [W/m²K], Wasserdampfdurchlasskoeffizient, GWP [kg CO2-eq./m³],
# Kosten [€/m³], Oberflaecheneignung [0/1], Eignung Bauteil innen [0/1], spez. Wärmekapazität c [J/kgK], Name
Materialdaten = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, "Leer"],
    [2400, 0.05, 0.4, 25, 2.5, 0.13, 100, 249.17, 200, 0, 1, 880, "Stahlbeton"], # Orientierung: Beton armiert 2% aus Ubakus, Willems
    [450, 0.01, 0.35, 18, 0.13, 0.13, 35, 101.5, 400, 1, 1, 1600, "Vollholz Fichte"], # Fichte, Ubakus
    [700, 0.01, 0.35, 55, 0.18, 0.13, 100, 24, 700, 1, 1, 1600, "Laub-Vollholz (Buche)"], # Konstruktionsholz 700 kg/m³, Willems; GWP : https://events.forum-holzbau.com/pdf/IHF_13_Dolezal.pdf
    [2000, 0.04, 0.49, 25, 0.96, 0.13, 10, 543.3, 400, 0, 1, 1000, "Vollziegel"], # Ubakus, Willems: Vollziegel 2000 kg/m³
    [900, 0.125, 0.49, 11, 0.37, 0.13, 10, 169, 165, 0, 1, 1000, "Hochlochziegel"], # Willems: Hochlochziegel 900 kg/m³, Oekobaudat
    [605, 0.125, 0.49, 3, 0.1, 0.13, 10, 144.4, 300, 0, 1, 1000, "Dämmziegel"], # Ubakus, Poroton T10
    [20, 0.02, 0.3, 0.00001, 0.04, 0.13, 1, 71, 92, 0, 1, 830, "Mineralwolle"], # Ubakus, Willems, Mineralwolle WLG040
    [38, 0.02, 0.3, 0.00001, 0.04, 0.13, 2, 13.9, 150, 0, 1, 2100, "Holzfaserdämmstoff"], # Ubakus, Holzfaser-Einblasdämmung
    [30, 0.02, 0.3, 0.001, 0.032, 0.13, 50, 145, 105, 0, 1, 1450, "EPS"], # EPS (WLG032), Willems
    [1200, 0.0001, 0.001, 0.00001, 0.22, 0.13, 1000000, 5707, 6000, 0, 1, 1700, "Dampfbremse PE-Folie"], # Orientierung: Ubakus Dampfbremse
    [1000, 0.001, 0.05, 0.01, 0.35, 0.13, 10, 132.1, 3000, 1, 0, 1090, "Gipsputz"], # Ubakus Gipsputz, Oekobaudat Gipsputz
    [680, 0.0125, 0.0125, 4, 0.25, 0.13, 7, 198.3, 400, 1, 1, 960, "GK-Platte"], # Ubakus Gipskartonplatte
    [7850, 0.001, 0.05, 250, 50, 0.13, 1000000, 15310, 6280, 0, 1, 800, "Stahlblech"], # Ubakus Baustahl
    [650, 0.005, 0.4, 25, 0.13, 0.13, 50, 185, 300, 1, 1, 1700, "OSB-Platte"], # Ubakus, Willems OSB-Platte
    [1450, 0.0125, 0.05, 3, 0.353, 0.13, 5, 450, 1800, 1, 1, 1100, "Lehmbauplatte"], # Orientierung: Conluto Lehmbauplatte aus Ubakus
    [1500, 0.05, 0.49, 3, 0.66, 0.13, 5, 101, 350, 0, 1, 1000, "Lehmziegel"], # Orientierung: Lehmsteine Ubakus, Preis: Geschätzt anhand günstiger Lehmziegel
    [1, 0.01, 0.15, 0, 0.134, 0.0001, 1, 0, 0, 0, 1, 1000, "Luftschicht (ruhend)"],
    [1, 0.01, 0.15, 0, 10, 0.0001, 1, 0, 0, 0, 1, 1000, "Luftschicht (hinterlüftet)"]
]
Materialien = len(Materialdaten)
Materialparameter = len(Materialdaten[0]) - 1
print(f"{Materialien} Materialien mit jeweils {Materialparameter} Eigenschaften sind definiert.")

Bauteileigenschaften = ['Gesamtdicke [m]', 'Gewicht [kg/m²]', 'Tragfaehigkeit [MN/m]', 'U-Wert [W/m²K]', 'GWP [kg CO₂-eq/m²]', 'Materialkosten [Euro/m²]', 'Feuchte [Ja/Nein]', 'Oberfläche [Ja/Nein]', 'Innenaufbau [Ja/Nein]', 'Waermekapazitaet [J/m²K]']
Eigenschaften = len(Bauteileigenschaften)
print(f"{Eigenschaften} Bauteileigenschaften werden unterschieden.")

19 Materialien mit jeweils 12 Eigenschaften sind definiert.
10 Bauteileigenschaften werden unterschieden.


# Funktionen, um Bauteileigenschaften aus Schichtaufbauten zu bestimmen:
Diese Funktionen berechnen die Eigenschaften Dicke, Gewicht, tragfähigkeit, U-Wert, GWP, Kosten, Tauwasseranfall, Oberflächeneignung außen und Innen und Verwendung korrekter Materialien im Innenbereich für jedes generierte Bauteil. Es wird eine Liste aller Eigenschaften erzeugt.

In [2]:
import numpy as np
import math

def Dicke(Aufbau):
  dicke = 0
  for i in Aufbau:
    dicke = dicke+i[1]
  return dicke

def Gewicht(Aufbau):
  gewicht = 0
  for i in Aufbau:
    gewicht = gewicht + i[1]*Materialdaten[i[0]][0]
  return gewicht

def Tragfaehigkeit(Aufbau):
  trag = 0
  for i in Aufbau:
    trag = trag + i[1]*Materialdaten[i[0]][3]
  return trag

def Uwert(Aufbau):
  uwert = 1/Materialdaten[Aufbau[0][0]][5] # Wärmeübergangswiderstand innen
  for i in Aufbau:
    uwert = 1/((1/uwert)+(i[1]/Materialdaten[i[0]][4]))
  uwert = 1/(1/uwert+0.307*Materialdaten[i[0]][5])
  return uwert

def GWP(Aufbau):
  gwp = 0
  for i in Aufbau:
    gwp = gwp + i[1]*Materialdaten[i[0]][7]
  return gwp

def Kosten(Aufbau):
  kosten = 0
  for i in Aufbau:
    kosten = kosten + i[1]*Materialdaten[i[0]][8]
  return kosten

def Glaser(Aufbau, Innentemperatur, Aussentemperatur, RLF_i, RLF_a):
# This function calculates the temperature and vapor pressure profiles across each layer of the construction component and checks if condensation occurs at any interface.
# The glaser_method function returns True if condensation (moisture accumulation) is detected and False otherwise.

    def saturation_vapor_pressure(T):
        # Tetens formula for saturation vapor pressure (Pa)
        return 610.78 * np.exp(T / (T + 238.3) * 17.2694)

    def vapor_pressure(T, RH):
        # Actual vapor pressure (Pa)
        return RH / 100.0 * saturation_vapor_pressure(T)

    # Indoor and outdoor conditions
    pi = vapor_pressure(Innentemperatur, RLF_i)
    pe = vapor_pressure(Aussentemperatur, RLF_a)

    # Calculate thermal resistance and vapor resistance for each layer
    Rth = [i[1] / Materialdaten[i[0]][4] for i in Aufbau]
    Rv = [i[1] * Materialdaten[i[0]][6] for i in Aufbau]

    # Total thermal and vapor resistance
    Rth_total = sum(Rth)
    Rv_total = sum(Rv)

    # Temperature and vapor pressure profiles
    T = [Innentemperatur]
    p = [pi]

    for i in range(len(Aufbau)):
        T_next = Innentemperatur - (Innentemperatur - Aussentemperatur) * sum(Rth[:i+1]) / Rth_total
        T.append(T_next)

        p_next = pi - (pi - pe) * sum(Rv[:i+1]) / Rv_total
        p.append(p_next)

    # Check for condensation at each interface
    for i in range(1, len(T)):
        if p[i] > saturation_vapor_pressure(T[i]):
            return 1

    return 0

def Oberflaeche(Aufbau):
  Eignung = 0
  if Materialdaten[Aufbau[0][0]][9] == 1 and Materialdaten[Aufbau[-1][0]][9] == 1:
    Eignung = 1
  return Eignung

def EignungInnen(Aufbau):
  Eignung = 1
  for i in range(1, len(Aufbau)-2):
    if Materialdaten[Aufbau[i][0]][10] == 0:
      Eignung = 0
  return Eignung

def Waermekapazitaet(Aufbau):
  c = 0
  for i in Aufbau:
    c = c + i[1]*Materialdaten[i[0]][11]*Materialdaten[i[0]][0]
  return c

def Schalldaemmass(Aufbau): # Diese Funktion spiegelt die Realität momentan noch nicht ausreichend wider. Es werden lediglich annähernde Berechnungsverfahren nach Recknagel (2022) angewendet.
  f = 140 # zu betrachtende Frequenz
  K = -16 # Konstante in der Formel für mehrschalige Aufbauten
  R = 69
  mehrschalig = 0
  # Feststellen, ob es sich um einen mehrschaligen Aufbau handelt
  # Trennschichten haben eine geringe Dichte (Luft oder leichter Dämmstoff) und zählen nicht als solche, wenn sie ganz innen oder außen sind
  Trennschicht = 0
  for i in range(1, len(Aufbau)-1):
    if Materialdaten[Aufbau[i][0]][0] < 50:
      mehrschalig = 1
      Trennschicht = i
    i = i+1

  mass = 0
  for i in Aufbau:
    mass = mass + i[1]*Materialdaten[i[0]][0]

  if mehrschalig == 1:
    R = 60*math.log10(f) + 40*math.log10(mass) + 20*math.log10(Aufbau[Trennschicht][1]) + K
    print("mehrschalig")
  else:
    R = 20*math.log10(f) + 20*math.log10(mass) - 47
    print("nicht mehrschalig")

  return R

########## Folgende Funktion stellt beispielhaft dar, wie weitere, z.B. akustische, Eigenschaften berechnet werden könnten. ###################
########## Sie ist noch nicht vollständig und die resultierenden Daten fließen nicht in die Datenbank ein. ####################################
def Schalldaemmass2(Aufbau): # Diese Funktion spiegelt die Realität momentan noch nicht ausreichend wider. Es werden lediglich annähernde Berechnungsverfahren nach Recknagel (2022) angewendet.
    frequenzen = [100, 125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000]  # Terzbänder
    K = -16  # Konstante in der Formel für mehrschalige Aufbauten
    R_ges = 0
    mehrschalig = False
    Trennschicht = 0

    # Feststellen, ob es sich um einen mehrschaligen Aufbau handelt
    for i in range(1, len(Aufbau)-1):
        if Materialdaten[Aufbau[i][0]][0] < 50:
            mehrschalig = True
            Trennschicht = i

    mass = sum(schicht[1] * Materialdaten[schicht[0]][0] for schicht in Aufbau)

    # Berechnung des Schalldämmmaßes für jede Frequenz
    for f in frequenzen:
        if mehrschalig:
            R_f = 60 * math.log10(f) + 40 * math.log10(mass) + 20 * math.log10(Aufbau[Trennschicht][1]) + K
        else:
            R_f = 20 * math.log10(f) + 20 * math.log10(mass) - 47
        R_ges += R_f

    # Mittelwert der Schalldämm-Maße über die Frequenzen
    R_mittel = R_ges / len(frequenzen)

    return R_mittel

def berechneEigenschaften(Aufbau):
  global Innentemperatur
  global Aussentemperatur
  global RLF_i
  global RLF_a
  Aufbau = [i for i in Aufbau if i[0] != 0] # delete "empty" layers of the Aufbau (Material 0 = "Leer")
  if Aufbau == []: # if all layers were empty, return empty values (or rather: Really bad values to suppress the algorithm for choosing this path of least resistance)
    return [0,0,0,10000,10000,10000,0,0,0]

  Eigenschaften = []
  Eigenschaften.append(Dicke(Aufbau))
  Eigenschaften.append(Gewicht(Aufbau))
  Eigenschaften.append(Tragfaehigkeit(Aufbau))
  Eigenschaften.append(Uwert(Aufbau))
  Eigenschaften.append(GWP(Aufbau))
  Eigenschaften.append(Kosten(Aufbau))
  Eigenschaften.append(Glaser(Aufbau, Innentemperatur, Aussentemperatur, RLF_i, RLF_a))
  Eigenschaften.append(Oberflaeche(Aufbau))
  Eigenschaften.append(EignungInnen(Aufbau))
  Eigenschaften.append(Waermekapazitaet(Aufbau))
  #print(Schalldaemmass2(Aufbau)) # Zu einem späteren Zeitpunkt könnten akustische Eigenschaften ergänzt werden
  return Eigenschaften

# Einen Testaufbau zur Überprüfung erzeugen
Testaufbau = [[1, 0.2], [6, 0.05], [1, 0.4], [0, 0.1], [0, 0.0]]
print(berechneEigenschaften(Testaufbau))

[0.65, 1470.25, 15.15, 1.0990097921772481, 156.722, 135.0, 1, 0, 1, 1297450.0]


# Funktion, um Bauteilaufbauten zu generieren.
Ziel ist es, eine Datenbank zu generieren, die Bauteilaufbauten mit realistischen Eigenschaften enthält. Dieser Code generiert die Datenbank und speichert sie im unten definierten Ordner ab. Zu beachten: Zielordner korrekt unten im Code eintragen!

In [3]:
import random
import pprint
import pandas as pd
import numpy as np
#from google.colab import drive #Nur bei Verwendung mit Google Colab notwendig
from datetime import datetime
import copy
from tqdm import tqdm

def generiereEigenschaften(minS, S, Materialien, Materialdaten):
    # Diese Funktion generiert einen Schichtaufbau mit einer zufälligen Anzahl von Schichten im festgelegten Rahmen
    # Anschließend werden die Eigenschaften berechnet
    # Erfüllen diese die Randbedingungen, wird der Bauteilaufbau ausgegeben
  while True:
    Aufbau = []
    MatProp = [0,0]
    for i in range(0, S):
      if i >= minS and random.uniform(0, 1) < 0.7 and len(Aufbau) == i: # Bauteilaufbauten erzeugen; ab der min_Schichten nimmt Wahrscheinlichkeit für eine weitere Schicht ab.
          nr = random.randint(1, Materialien - 1)
          MatProp = [nr, random.uniform(Materialdaten[nr][1], Materialdaten[nr][2])]
      elif i < minS:
          nr = random.randint(1, Materialien - 1)
          MatProp = [nr, random.uniform(Materialdaten[nr][1], Materialdaten[nr][2])]
      else:
          break
      Aufbau.append(MatProp)
    # Unpassende / unrealistische Bauteilaufbauten rausfiltern, z.B. zu dick, zu dünn
    Eig = berechneEigenschaften(Aufbau)
    if Eig[0] <= max_Dicke and Eig[0] >= min_Dicke and Eig[1] <= max_Gewicht and Eig[2] <= max_Tragfaehigkeit and Eig[2] >= min_Tragfaehigkeit and Eig[3]<=max_Uwert and Eig[4] <= max_GWP and Eig[5] <= max_Kosten and (Eig[6] < 1 or Feuchte_beachten == False) and (Eig[7]>0 or Oberflaechen_beachten == False) and (Eig[8]>0 or Innenaufbau_beachten == False):
      break

    # Eigenschaften ausgeben, wenn Randbedingungen erfüllt sind
  Bauteileigenschaften = berechneEigenschaften(Aufbau)
  for i in Aufbau:
      # Materialnamen statt -Nummern speichern
    i[0] = Materialdaten[i[0]][12]
  return [Aufbau, Bauteileigenschaften]

def normalisieren(Eigenschaften):
    # Normalisierung der Eigenschaften. Normalisierte Datenbank wird derzeit nicht abgespeichert.
  normalisiert = []
  global max_Werte
  for i in range(0, len(Eigenschaften)):
    normalisiert.append(Eigenschaften[i]/max_Werte[i])
  return normalisiert

def denormalisieren(normalEigenschaften):
    # Denormalisierung der Eigenschaften. Wird derzeit nicht verwendet.
  denormalisiert = []
  global max_Werte
  for i in range(0, len(normalEigenschaften)):
    denormalisiert.append(normalEigenschaften[i]*max_Werte[i])
  return denormalisiert


# so viele Bauteile erzeugen, die die zuvor definierten Bedingungen erfüllen, wie Anzahl_Bauteile vorgibt
Bauteil_Datensatz = []
for i in tqdm(range(0, Anzahl_Bauteile), desc="Generiere Datensätze...", unit="part"):
    Bauteil_Datensatz.append(generiereEigenschaften(min_Schichten, Schichten, Materialien, Materialdaten))
print("Datensätze vollständig generiert. Verarbeiten...")
Bauteil_Datensatz_normalisiert = copy.deepcopy(Bauteil_Datensatz)
for i in Bauteil_Datensatz_normalisiert:
  i[1] = normalisieren(i[1])

# Bereinigung der Ergebnisse. Da es Bauteile mit unterschiedlicher Anzahl von Schichten erzeugt hat, 
# werden Bauteile mit weniger Schichten mit leeren Schichten [0,0] aufgefüllt, um eine einheitliche Datenstruktur zu gewährleisten.
# verwendet dafür den nicht normalisierten Datensatz. Die Normalisierung wird durch das Notebook, das den ML-Algorithmus enthält, bereitgestellt.
max_length = Schichten
Datensatz_uniform = []
for item in Bauteil_Datensatz:
    uniform_part = item[0] + [[0, 0]] * (max_length - len(item[0]))
    flattened_row = [elem for sublist in uniform_part for elem in sublist]
    flattened_row.extend(item[1])
    Datensatz_uniform.append(flattened_row)

#pprint.pprint(Datensatz_uniform)
# Sichergehen, dass alle Reihen die korrekte Anzahl an Spalten hat, um Fehlern vorzubeugen
for row in Datensatz_uniform:
    while len(row) < Schichten*2+Eigenschaften:
        print("Unvollständiger Datensatz gefunden und ergänzt. Datenerzeugung überprüfen.")
        row.append(np.nan)

# Header für den Datensatz kreieren
headers = []
for i in range(1, Schichten + 1):
    headers.append(f'Material {i}')
    headers.append(f'Dicke {i}')
rest_headers = Bauteileigenschaften
headers.extend(rest_headers)

# DataFrame kreieren
df = pd.DataFrame(Datensatz_uniform, columns = headers)
# Save to CSV 
current_datetime = datetime.now().strftime('%Y%m%d_%H%M')
filename = f'{current_datetime}_BT_Data_uniform.csv'

############## ENTER FOLDER PATH HERE ######################################################################################################################################################
csv_path = f'C:/Users/emil-/Desktop/Dinge/{filename}' # Replace the folder path with the desired location (if using google colab: A Google Drive Location is required)
# Attention: Python only allows Slashes /, not backslashes \ in filenames
############################################################################################################################################################################################
df.to_csv(csv_path, index=False)

print(f"File saved at {csv_path}")


Generiere Datensätze...: 100%|███████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1854.29part/s]


Datensätze vollständig generiert. Verarbeiten...
File saved at C:/Users/emil-/Desktop/Dinge/20241017_1152_BT_Data_uniform.csv


# Generierte .csv-Dateien einlesen, falls nötig

In [4]:
# Define the path to the CSV file
# Decide if to use the latest file
UseLatestCSV = True
CSVname = '20240702_0908_BT_Data_normalized'
if not UseLatestCSV:
  csv_path = f'/content/drive/MyDrive/Masterarbeit_ML/Bauteildata/{CSVname}'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Display the first few rows of the DataFrame
print(df.head())

              Material 1   Dicke 1                  Material 2   Dicke 2  \
0             OSB-Platte  0.123305  Luftschicht (hinterlüftet)  0.031846   
1        Vollholz Fichte  0.128606               Lehmbauplatte  0.029295   
2               Gipsputz  0.046037        Dampfbremse PE-Folie  0.000738   
3               Gipsputz  0.011148                  Lehmziegel  0.241332   
4  Laub-Vollholz (Buche)  0.023343               Lehmbauplatte  0.023979   

             Material 3   Dicke 3 Material 4  Dicke 4 Material 5  Dicke 5  \
0       Vollholz Fichte  0.082773          0  0.00000          0      0.0   
1            OSB-Platte  0.253176          0  0.00000          0      0.0   
2       Vollholz Fichte  0.180555          0  0.00000          0      0.0   
3       Vollholz Fichte  0.250223          0  0.00000          0      0.0   
4  Luftschicht (ruhend)  0.132327   Gipsputz  0.04049          0      0.0   

   ... Gesamtdicke [m]  Gewicht [kg/m²]  Tragfaehigkeit [MN/m]  \
0  ...        